In [1]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/data/CiteSeer', name='CiteSeer')
print(len(dataset))
data = dataset[0]
data = data.to(device)
print(data)

1
Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


In [3]:
def eval_model(model, myprint=False):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    loss_function = torch.nn.CrossEntropyLoss().to(device)
    model.train()
    for epoch in range(400):
        optimizer.zero_grad()
        out = model(data)
        loss = loss_function(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        if myprint and epoch % 50 == 0:
            print("epoch {}, loss = {}".format(epoch, loss))
    model.eval()
    pred = model(data).argmax(dim=1)
    correct = (pred[data.val_mask] == data.y[data.val_mask]).sum()
    acc = int(correct) / int(data.val_mask.sum())
    if myprint:
        print(f'Accuracy: {acc:.4f}')
    return acc

In [4]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
    
class GCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = F.softmax(x, dim=1)

        return x

In [6]:
myGCN = GCN(dataset.num_node_features, dataset.num_classes)
eval_model(myGCN, myprint=True)

epoch 0, loss = 1.7914729118347168
epoch 50, loss = 1.4141508340835571
epoch 100, loss = 1.3998398780822754
epoch 150, loss = 1.4621336460113525
epoch 200, loss = 1.4432032108306885
epoch 250, loss = 1.4105377197265625
epoch 300, loss = 1.452028751373291
epoch 350, loss = 1.381689429283142
Accuracy: 0.6840


0.684

In [9]:
class GCN_tryhidden(torch.nn.Module):
    def __init__(self, hidden, num_node_features=dataset.num_node_features, num_classes=dataset.num_classes):
        super(GCN_tryhidden, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden)
        self.conv2 = GCNConv(hidden, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = F.softmax(x, dim=1)

        return x

In [10]:
hidden = [10, 20, 30, 40, 50]
accs = []
for h in hidden:
    myGCN = GCN_tryhidden(h)
    acc = eval_model(myGCN)
    print("hidden size = {}, validation accuracy = {:.2%}".format(h, acc))
    accs.append(acc)

hidden size = 10, validation accuracy = 67.40%
hidden size = 20, validation accuracy = 66.60%
hidden size = 30, validation accuracy = 67.80%
hidden size = 40, validation accuracy = 66.80%
hidden size = 50, validation accuracy = 66.80%
